## Some SQL for hh.spell exploration

*original code:
left join hh_spells with itself on next spell start more than old spell end
this will still format one old spell with multiple new spells *

CREATE TABLE c6.return_hh_indcase_spells AS 
SELECT sub.ch_dpa_caseid, sub.recptno,sub.ssn_hash,sub.oldBene_type, sub.newBene_type, sub.member_age,
sub.oldSpell_end, sub.newSpell_start, sub.returnSpell_gap,
CASE
 WHEN sub.returnSpell_gap <=0 THEN 0
 ELSE 1
 END AS "return_index"

FROM
(SELECT a.recptno, a.ch_dpa_caseid, a.ssn_hash,a.benefit_type AS oldBene_type,
a.end_date AS oldSpell_end, 
AGE(a.end_date, a.birth_date )AS member_age,
b.benefit_type AS newBene_type,
b.start_date AS newSpell_start,
( b.start_date- a.end_date ) AS returnSpell_gap
FROM idhs.hh_indcase_spells a
LEFT JOIN idhs.hh_indcase_spells b
ON (a.recptno=b.recptno) AND
(a.end_date<b.start_date)
)
sub;

* logic on left join hh_spell with itself to find out if a case has a future spell*
SELECT a.recptno, a.ch_dpa_caseid, a.ssn_hash,a.benefit_type AS oldBene_type,
a.end_date AS oldSpell_end, 
AGE(a.end_date, a.birth_date )AS member_age,
b.benefit_type AS newBene_type,
b.start_date AS newSpell_start,
CASE
WHEN b.start_date IS NULL THEN 1000000
ELSE ( b.start_date- a.end_date ) 
END AS returnSpell_gap
FROM idhs.hh_indcase_spells a
LEFT JOIN idhs.hh_indcase_spells b
ON (a.recptno=b.recptno) AND
(a.end_date<b.start_date)

* logic on given case link to future spell date, aggregate to the nearest spell*
SELECT sub.ch_dpa_caseid, sub.recptno,sub.ssn_hash,sub.oldBene_type, sub.newBene_type, sub.member_age,
sub.oldSpell_end, min(sub.newSpell_start) as next_spell_start_date, min(sub.returnSpell_gap) as gap,
CASE
 WHEN min(sub.returnSpell_gap) <1000000 THEN 1
 ELSE 0
 END AS "return_index",
 quarter and year calculation
 group by 1,2,3,4,5,6,7;

*logic on create a next quarter and next year variable based on end_date*
//extract(month from end_date) as month,extract(year from end_date) as year, 

case
when extract(month from end_date) in (1,2,3) then 2
when extract(month from end_date) in (4,5,6) then 3
when extract(month from end_date) in (7,8,9) then 4
else 1
end as wage_quarter
case
when extract(month from end_date)>9 then extract(year from end_date)+1
else extract(year from end_date)
end as wage_year


/*tp2 */
case
when extract(month from oldSpell_end) in (1,2,3) then 3
when extract(month from oldSpell_end) in (4,5,6) then 4
when extract(month from oldSpell_end) in (7,8,9) then 1
else 2
end as quarter_tp2,
case
when extract(month from oldSpell_end)>6 then extract(year from oldSpell_end)+1
else extract(year from oldSpell_end)
end as year_tp2,
/*tp1 */
case
when extract(month from oldSpell_end) in (1,2,3) then 2
when extract(month from oldSpell_end) in (4,5,6) then 3
when extract(month from oldSpell_end) in (7,8,9) then 4
else 1
end as quarter_tp1,
case
when extract(month from oldSpell_end)>9 then extract(year from oldSpell_end)+1
else extract(year from oldSpell_end)
end as year_tp1,
/*tm1 */
case
when extract(month from oldSpell_end) in (4,5,6) then 1
when extract(month from oldSpell_end) in (7,8,9) then 2
when extract(month from oldSpell_end) in (10,11,12) then 3
else 4
end as quarter_tm1,
case
when extract(month from oldSpell_end)<4 then extract(year from oldSpell_end)-1
else extract(year from oldSpell_end)
end as year_tm1,
/*tm2 */
case
when extract(month from oldSpell_end) in (7,8,9) then 1
when extract(month from oldSpell_end) in (10,11,12) then 2
when extract(month from oldSpell_end) in (1,2,3) then 3
else 4
end as quarter_tm2,
case
when extract(month from oldSpell_end)<7 then extract(year from oldSpell_end)-1
else extract(year from oldSpell_end)
end as year_tm2,
/*tm3*/
case
when extract(month from oldSpell_end) in (10,11,12) then 1
when extract(month from oldSpell_end) in (1,2,3) then 2
when extract(month from oldSpell_end) in (4,5,6) then 3
else 4
end as quarter_tm3,
case
when extract(month from oldSpell_end)<10 then extract(year from oldSpell_end)-1
else extract(year from oldSpell_end)
end as year_tm3,
/*tm4 */
case
when extract(month from oldSpell_end) in (1,2,3) then 1
when extract(month from oldSpell_end) in (4,5,6) then 2
when extract(month from oldSpell_end) in (7,8,9) then 3
else 4
end as quarter_tm4,
case
when extract(month from oldSpell_end)<13 then extract(year from oldSpell_end)-1
else extract(year from oldSpell_end)
end as year_tm4,

*final code with update on quarter/year on 6/19/2017
DROP TABLE IF EXISTS c6.return_hh_indcase_spells ;
CREATE TABLE c6.return_hh_indcase_spells AS 
SELECT sub.ch_dpa_caseid, sub.recptno,sub.member_age,sub.oldBene_type,
sub.oldSpell_end, min(sub.newSpell_start) as next_spell_start_date, min(sub.returnSpell_gap) as gap,
CASE
 WHEN min(sub.returnSpell_gap) <1000000 THEN 1
 ELSE 0
 END AS "return_index",
  /*tp4 */
case
when extract(month from oldSpell_end) in (1,2,3) then 1
when extract(month from oldSpell_end) in (4,5,6) then 2
when extract(month from oldSpell_end) in (7,8,9) then 3
else 4
end as quarter_tp4,
case
when extract(month from oldSpell_end)>0 then extract(year from oldSpell_end)+1
else extract(year from oldSpell_end)
end as year_tp4,
 /*tp3 */
case
when extract(month from oldSpell_end) in (1,2,3) then 4
when extract(month from oldSpell_end) in (4,5,6) then 1
when extract(month from oldSpell_end) in (7,8,9) then 2
else 3
end as quarter_tp3,
case
when extract(month from oldSpell_end)>3 then extract(year from oldSpell_end)+1
else extract(year from oldSpell_end)
end as year_tp3,
 /*tp2 */
case
when extract(month from oldSpell_end) in (1,2,3) then 3
when extract(month from oldSpell_end) in (4,5,6) then 4
when extract(month from oldSpell_end) in (7,8,9) then 1
else 2
end as quarter_tp2,
case
when extract(month from oldSpell_end)>6 then extract(year from oldSpell_end)+1
else extract(year from oldSpell_end)
end as year_tp2,
/*tp1 */
case
when extract(month from oldSpell_end) in (1,2,3) then 2
when extract(month from oldSpell_end) in (4,5,6) then 3
when extract(month from oldSpell_end) in (7,8,9) then 4
else 1
end as quarter_tp1,
case
when extract(month from oldSpell_end)>9 then extract(year from oldSpell_end)+1
else extract(year from oldSpell_end)
end as year_tp1,
/*tm1 */
case
when extract(month from oldSpell_end) in (4,5,6) then 1
when extract(month from oldSpell_end) in (7,8,9) then 2
when extract(month from oldSpell_end) in (10,11,12) then 3
else 4
end as quarter_tm1,
case
when extract(month from oldSpell_end)<4 then extract(year from oldSpell_end)-1
else extract(year from oldSpell_end)
end as year_tm1,
/*tm2 */
case
when extract(month from oldSpell_end) in (7,8,9) then 1
when extract(month from oldSpell_end) in (10,11,12) then 2
when extract(month from oldSpell_end) in (1,2,3) then 3
else 4
end as quarter_tm2,
case
when extract(month from oldSpell_end)<7 then extract(year from oldSpell_end)-1
else extract(year from oldSpell_end)
end as year_tm2,
/*tm3*/
case
when extract(month from oldSpell_end) in (10,11,12) then 1
when extract(month from oldSpell_end) in (1,2,3) then 2
when extract(month from oldSpell_end) in (4,5,6) then 3
else 4
end as quarter_tm3,
case
when extract(month from oldSpell_end)<10 then extract(year from oldSpell_end)-1
else extract(year from oldSpell_end)
end as year_tm3,
/*tm4 */
case
when extract(month from oldSpell_end) in (1,2,3) then 1
when extract(month from oldSpell_end) in (4,5,6) then 2
when extract(month from oldSpell_end) in (7,8,9) then 3
else 4
end as quarter_tm4,
case
when extract(month from oldSpell_end)<13 then extract(year from oldSpell_end)-1
else extract(year from oldSpell_end)
end as year_tm4
FROM
(SELECT a.recptno, a.ch_dpa_caseid, a.ssn_hash,a.benefit_type AS oldBene_type,
a.end_date AS oldSpell_end, 
AGE(a.end_date, a.birth_date )AS member_age,
b.benefit_type AS newBene_type,
b.start_date AS newSpell_start,
CASE
WHEN b.start_date IS NULL THEN 1000000
ELSE ( b.start_date- a.end_date ) 
END AS returnSpell_gap
FROM idhs.hh_indcase_spells a
LEFT JOIN idhs.hh_indcase_spells b
ON (a.recptno=b.recptno) AND
(a.end_date<b.start_date)
)
sub
group by 1,2,3,4,5
order by 1,2,4,5;

In [ ]:
## 07/04/2017
# code above revision: 1:seperate year quarter out 2. table to construct is changed to no-grant
DROP TABLE IF EXISTS c6.return_hh_indcase_spells ;
CREATE TABLE c6.return_hh_indcase_spells AS
SELECT sub.recptno,sub.oldSpell_end,sub.oldBene_type,
min(sub.ch_dpa_caseid) AS ch_dpa_caseid,avg(sub.member_age) AS member_age, min(sub.newSpell_start) as next_spell_start_date, min(sub.returnSpell_gap) as gap,
CASE
 WHEN min(sub.returnSpell_gap) <1000000 THEN 1
 ELSE 0
 END AS "return_index",
CASE When count(*)=1 AND min(sub.returnSpell_gap)=1000000 THEN 0
ELSE count(*)
END AS "total_future_spell"
FROM
(SELECT a.recptno, a.ch_dpa_caseid,a.benefit_type AS oldBene_type,
a.end_date AS oldSpell_end, 
AGE(a.end_date, a.birth_date )AS member_age,
b.benefit_type AS newBene_type,
b.start_date AS newSpell_start,
CASE
WHEN b.start_date IS NULL THEN 1000000
ELSE ( b.start_date- a.end_date ) 
END AS returnSpell_gap
FROM c6.hh_indcase_spells_nogrant a
LEFT JOIN c6.hh_indcase_spells_nogrant b
ON (a.recptno=b.recptno) AND
(a.end_date<b.start_date)
 order by 1,3,4,6,7
)
sub
group by 1,2,3
order by 1,2,3
;

In [ ]:
DROP TABLE IF EXISTS c6.yq_return_hh_indcase_spells ;
CREATE TABLE c6.yq_return_hh_indcase_spells AS
SELECT recptno,oldSpell_end,
/*tp4 */
case
when extract(month from oldSpell_end) in (1,2,3) then 1
when extract(month from oldSpell_end) in (4,5,6) then 2
when extract(month from oldSpell_end) in (7,8,9) then 3
else 4
end as quarter_tp4,
case
when extract(month from oldSpell_end)>0 then extract(year from oldSpell_end)+1
else extract(year from oldSpell_end)
end as year_tp4,
 /*tp3 */
case
when extract(month from oldSpell_end) in (1,2,3) then 4
when extract(month from oldSpell_end) in (4,5,6) then 1
when extract(month from oldSpell_end) in (7,8,9) then 2
else 3
end as quarter_tp3,
case
when extract(month from oldSpell_end)>3 then extract(year from oldSpell_end)+1
else extract(year from oldSpell_end)
end as year_tp3,
 /*tp2 */
case
when extract(month from oldSpell_end) in (1,2,3) then 3
when extract(month from oldSpell_end) in (4,5,6) then 4
when extract(month from oldSpell_end) in (7,8,9) then 1
else 2
end as quarter_tp2,
case
when extract(month from oldSpell_end)>6 then extract(year from oldSpell_end)+1
else extract(year from oldSpell_end)
end as year_tp2,
/*tp1 */
case
when extract(month from oldSpell_end) in (1,2,3) then 2
when extract(month from oldSpell_end) in (4,5,6) then 3
when extract(month from oldSpell_end) in (7,8,9) then 4
else 1
end as quarter_tp1,
case
when extract(month from oldSpell_end)>9 then extract(year from oldSpell_end)+1
else extract(year from oldSpell_end)
end as year_tp1,
/*tm1 */
case
when extract(month from oldSpell_end) in (4,5,6) then 1
when extract(month from oldSpell_end) in (7,8,9) then 2
when extract(month from oldSpell_end) in (10,11,12) then 3
else 4
end as quarter_tm1,
case
when extract(month from oldSpell_end)<4 then extract(year from oldSpell_end)-1
else extract(year from oldSpell_end)
end as year_tm1,
/*tm2 */
case
when extract(month from oldSpell_end) in (7,8,9) then 1
when extract(month from oldSpell_end) in (10,11,12) then 2
when extract(month from oldSpell_end) in (1,2,3) then 3
else 4
end as quarter_tm2,
case
when extract(month from oldSpell_end)<7 then extract(year from oldSpell_end)-1
else extract(year from oldSpell_end)
end as year_tm2,
/*tm3*/
case
when extract(month from oldSpell_end) in (10,11,12) then 1
when extract(month from oldSpell_end) in (1,2,3) then 2
when extract(month from oldSpell_end) in (4,5,6) then 3
else 4
end as quarter_tm3,
case
when extract(month from oldSpell_end)<10 then extract(year from oldSpell_end)-1
else extract(year from oldSpell_end)
end as year_tm3,
/*tm4 */
case
when extract(month from oldSpell_end) in (1,2,3) then 1
when extract(month from oldSpell_end) in (4,5,6) then 2
when extract(month from oldSpell_end) in (7,8,9) then 3
else 4
end as quarter_tm4,
case
when extract(month from oldSpell_end)<13 then extract(year from oldSpell_end)-1
else extract(year from oldSpell_end)
end as year_tm4
from 
(select recptno,oldSpell_end
from c6.return_hh_indcase_spells
group by recptno,oldSpell_end) a


*final code with update on features on 6/14/2017
/* create features*/
/* features from avishek's notebook*/
Our preliminary features are the following
n_spells (Aggregation): Total number of spells someone has had up until the end_date of current spell.
max/min/avg_preSpell_length (Aggregation): max/min/avg of spell length of spells someone has had up until the end_date of current spell.
spell_length: the length of current spell: end_date- start_date.

age (Transformation): The age feature is created by substracting the bdate_year with the current year of prediction.
edlevel (Binary): 0 if the person has less than a high school education and 1 if they are more than a high school education.
workexp (Binary): 0 if no work experience 1 if there is some sort of work experience
married (Binary): 1 if the person is married 0 if they are not.
gender: (Binary) 1(male) 2(female)
district: (Binary): 0 if downstate 1 if cook county 3.special purpose
race: 1. white 2. black 3.other
homeless: 0 not homeless, 1-3 have shelter 4 not have shelter
(foodstamp, tanf, granf): (Binary) 0 if the last benefit was not foodstamp, tanf or grantf, 1 if it was

/*spell_length and number of previous spells*/
/*07/07/2017*/
drop table if exists c6.pre_features_spells_info;
create table c6.pre_features_spells_info as
select c.recptno,c.benefit_type,c.end_date,c.spell_length,
count(*) -1 as n_preSpells,
max(c.preSpell_length) AS max_Spell_length,
MIN(c.preSpell_length) AS min_Spell_length,
AVG(c.preSpell_length) AS avg_Spell_length,
SUM(c.pre_foodstamp) AS total_foodstamp_utlnow,
SUM(c.pre_tanf) AS total_tanf_utlnow
FROM
(SELECT b.recptno,a.benefit_type AS preBene_type,
a.start_date as preSpell_start,a.end_date AS preSpell_end,(a.end_date-a.start_date) AS preSpell_length,
case when a.benefit_type = 'foodstamp' then 1 else 0 end pre_foodstamp,
case when a.benefit_type = 'tanf46' then 1 else 0 end pre_tanf,
b.benefit_type,
b.start_date,b.end_date,(b.end_date-b.start_date) AS spell_length,
case when b.benefit_type = 'foodstamp' then 1 else 0 end foodstamp,
case when b.benefit_type = 'tanf46' then 1 else 0 end tanf
FROM c6.hh_indcase_spells_nogrant a
RIGHT JOIN c6.hh_indcase_spells_nogrant b
ON (a.recptno=b.recptno) AND
(a.end_date<=b.end_date)
order by b.recptno,b.end_date,b.benefit_type) as c
group by 1,2,3,4

/*features from idhs.member_info: edlevel,martlst,workexp*/
drop table if exists c6.pre_features_member_info;
create table c6.pre_features_member_info as 
select c.ch_dpa_caseid,c.recptno,
case when c.edlevel in ('A', 'B', 'C', 'D', '1', '2', '3','E', 'F', '4') THEN 1
     when c.edlevel in ('G', 'H', 'V', '5') THEN 2
     when c.edlevel in ('W', 'X', 'Y', '6') THEN 3
     when c.edlevel in ('Z', 'P', '7') THEN 4
     else 0 end edlevel,
case when c.martlst in (0,1,3,4,5,6) then 0 
else 1 end marstat,
case when c.workexp in ('0','1') then 0 
else c.workexp end workexp
from (select a.ch_dpa_caseid,a.recptno,
           b.edlevel,b.workexp,b.martlst
from (select ch_dpa_caseid,recptno
from c6.return_hh_indcase_spells
group by ch_dpa_caseid,recptno) a 
left join idhs.member_info b
 on a.recptno = b.recptno and a.ch_dpa_caseid=b.ch_dpa_caseid) as c;  
 
/*features from idhs.assistance_case: district,race,homeless*/
drop table if exists c6.pre_features_assistance_case;
create table c6.pre_features_assistance_case as
select c.ch_dpa_caseid,
case when c.district >=10 and c.district <=115 then 0 
when c.district >=200 and c.district <=294 then 1
else 2 end district,
case when c.homeless='N' then 0 else 1 end homeless,
case when c.race=1 then 1
when c.race=2 then 2
else 3 end race
from (
select a.ch_dpa_caseid,
b.district,b.race,b.homeless
from 
(select ch_dpa_caseid
from c6.return_hh_indcase_spells
group by ch_dpa_caseid) a
left join idhs.assistance_case b
on a.ch_dpa_caseid=b.ch_dpa_caseid) as c

/*features from idhs.hh_member: sex,rootrace,hh_counts,foreignbn*/
/* note on hh_count compare to count(*) as num_people from idhs_member group by recptno: hh_count is always smaller or equal to num_people. I choose to use hh_count because num_people can get very big to 20+ etc. I think hh_count is visually more reasonable and like to hh closer */
/* code to explore the relationship is :
select distinct(c.compare)
from (
select a.recptno, a.hh_counts, b.num_people,
case when a.hh_counts= b.num_people then 1
when a.hh_counts< b.num_people then 0
else 2 end as "compare"
from
(select recptno,hh_counts
from idhs.hh_member) as a
join (select recptno,count(*) as num_people
from idhs.member
group by 1) as b
on a.recptno=b.recptno ) as c */

drop table if exists c6.pre_features_hh_member;
create table c6.pre_features_hh_member as
select c.recptno,
case when c.sex is NULL then 0
when c.sex=1 then 1
when c.sex=2 then 2
else 0 end sex,
case when c.rootrace NOT in (1,2,3,6,7) then 0 
else c.rootrace end rootrace,
case when c.foreignbn is NULL then 0 else c.foreignbn end foreignbn,
case when c.hh_counts is NULL then 0 else c.hh_counts end hh_counts
from
(select a.recptno,
       b.sex,b.rootrace,b.hh_counts,b.foreignbn
from (select recptno
from c6.return_hh_indcase_spells
group by recptno) a
left join 
idhs.hh_member b
on a.recptno = b.recptno 
order by 1 ) as c